In [6]:
from dotenv import load_dotenv
from typing import Annotated
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(
    model="llama-3.3-70b-versatile",
    base_url="https://api.groq.com/openai/v1",
    temperature=0
)

print(llm.invoke("Say hello").content)


Hello. How can I assist you today?


In [7]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [9]:
message = {"role": "user", "content": "Who walked on the moon first?"}
response = graph.invoke({"messages": [message]})

response["messages"]

[HumanMessage(content='Who walked on the moon first?', additional_kwargs={}, response_metadata={}, id='e45b3109-8d91-485f-8d7b-862a1959ef67'),
 AIMessage(content='Neil Armstrong was the first person to walk on the moon. He stepped out of the lunar module Eagle and onto the moon\'s surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 42, 'total_tokens': 117, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.052897483, 'prompt_time': 0.001952417, 'completion_time': 0.141363634, 'total_time': 0.143316051}, 'model_provider': 'openai', 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'id': 'chatcmpl-f6108cec-d6c1-4e2c-a1bf-77e305e8314a', 'service_tier': 'on_demand', 'fi

In [11]:
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in ["exit", "quit"]:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

Bot: Neil Armstrong
Bot: Valentina Tereshkova
Bot: 1963
Bot: 1. First person to land on the moon: Neil Armstrong (1969)
2. First woman in space: Valentina Tereshkova (1963)
Bot: To date, no woman has walked on the Moon. The first person to walk on the Moon was Neil Armstrong in 1969, but no woman has yet had the opportunity to set foot on the lunar surface.

However, the first woman to fly in space was Valentina Tereshkova in 1963.
